In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
from scipy import optimize
import numba as nb
from mwc_class import getMVCs
from mwc_functions import *
from optimization_functions import *

In [2]:
gerdata = getMVCs('germany.csv', save_results=False)
results = gerdata.run_pipeline()


In [3]:
transformed_dataframe = results.get("transformed_dataframe")
parties_in_year = results.get("parties_in_year")
n_in_year = results.get("n_in_year")
totalseats_in_year = results.get("totalseats_in_year")
coalition_dict = results.get("coalition_dict")
winning_coal_dict = results.get("winning_coal_dict")
minimal_winning_coalitions = results.get("minimal_winning_coalitions")
maximal_losing_coalitions = results.get("maximal_losing_coalitions")
unique_tying_coalitions = results.get("unique_tying_coalitions")

In [4]:

all_year_dfs = create_all_year_dfs(winning_coal_dict, parties_in_year)

Example

In [5]:
year = '2002' 
df_for_year = all_year_dfs.get(year)
print(df_for_year)


                         CDU  CSU  FDP  G -  PDS  SPD  Winning
                           0    0    0    0    0    0        0
CDU                        1    0    0    0    0    0        0
CSU                        0    1    0    0    0    0        0
FDP                        0    0    1    0    0    0        0
G -                        0    0    0    1    0    0        0
...                      ...  ...  ...  ...  ...  ...      ...
CDU+CSU+FDP+PDS+SPD        1    1    1    0    1    1        1
CDU+CSU+G -+PDS+SPD        1    1    0    1    1    1        1
CDU+FDP+G -+PDS+SPD        1    0    1    1    1    1        1
CSU+FDP+G -+PDS+SPD        0    1    1    1    1    1        1
CDU+CSU+FDP+G -+PDS+SPD    1    1    1    1    1    1        1

[64 rows x 7 columns]


apparently the next cell is not good practice since python does not like global variables. 
Later everything will be written in terms of dicts, for now, this will do.

In [6]:
for year, df in all_year_dfs.items():
    globals()[f'df_of_{year}'] = df

now it gets interesting

In [7]:
# Example usage:
constraints_df = generate_constraints_df(df_for_year)

In [8]:
print(len(constraints_df))
print(constraints_df.head())

1030
   w_CDU  w_CSU  w_FDP  w_G -  w_PDS  w_SPD
0      1      0      0      0      0      0
1      0      1      0      0      0      0
2      0      0      1      0      0      0
3      0      0      0      1      0      0
4      0      0      0      0      1      0


testing whether all constraints are found: 

there will be $2^n$ coalitions with $|W|*(2^n-|W|)+n$ constraints
where W are all winning coalitions 

see "The Minimum Sum Representation as an Index of Voting Power" Josep Freixas and Serguei Kaniovski (October 17, 2013)

In [13]:
constraints_df = generate_constraints_df(df_for_year)  
result = verify_conditions('2002', winning_coal_dict, constraints_df, n_in_year)
print(result)

Correct


In [25]:
constraints_df.shape[1]

6

In [26]:
import numpy as np
from scipy.optimize import LinearConstraint

def get_lin_cons(constraints_df):
    # creates constraints usable for milp
    # gets matrix A from constraints_df
    # first n constraints are lhs>=0, 
    # remaining constraints are lhs>0, since lhs will always be integer-valued this is equivalent to lhs>=1 
    # returns lin_constraint element   
    A = constraints_df.to_numpy()
    lbnd = np.zeros(len(constraints_df)) # non-negativity constraints 
    lbnd[constraints_df.shape[1]:] = 1 #set all remaining lower bounds to 1
    upbnd = np.full(len(constraints_df), np.inf) #no upper bound 

    lin_cons = LinearConstraint(A, lbnd, upbnd)

    return lin_cons


In [27]:
lin_cons = get_lin_cons(constraints_df)


In [29]:
from scipy.optimize import milp
mvw = optimize.milp(np.full(n_in_year['2002'],1), integrality=np.full(n_in_year['2002'],1), constraints=lin_cons)

In [30]:
def mvw_to_parties(optimizer_results, parties_in_year, year):
    ## takes as input milp.x object and party dict as well as year. 
    ## zips optimal weights to repsective parties 
    optimized_seats = {party: result for party, result in zip(parties_in_year[year], optimizer_results)}

    return optimized_seats

In [34]:
ger2002_mvw=mvw_to_parties(mvw.x,parties_in_year,'2002')

In [36]:
import itertools

def test_mvw_1(optimized_seats):
    ## creates dict of winning_coal_dict form, used later to ensure equivalency of games 
    ## takes in dict from mvw_to_parties and uses same logic as coalition_combinatorics_generator and win_coals but for dicts
    
    mw_winning_coal_dict = {}
    total_seats = sum(optimized_seats.values()) #new Q
    parties = list(optimized_seats.keys())

    # Generate all unique coalitions
    mw_winning_coal_dict[''] = 0
    for r in range(1, len(parties) + 1):
        for combo in itertools.combinations(parties, r):
            coalition = '+'.join(combo) #name of coalition 
            coalition_seats = sum(optimized_seats[party] for party in combo) #seats is sum of their weights 
            mw_winning_coal_dict[coalition] = 1 if coalition_seats > (total_seats / 2) else 0 #indicate winning or losing (again, with strict inequality)

    return mw_winning_coal_dict


In [37]:
ger2002mv_dict = test_mvw_1(ger2002_mvw)

In [38]:
def test_mvw_2(year, winning_coal_dict, mw_winning_coal_dict):
    yearly_wins = {coalition: value for (yr, coalition), value in winning_coal_dict.items() if yr == year} # bit convoluted since I need 'year' twice, basically just separates the relevant year from the winning_coal_dict

    # Compare 
    for coalition, value in yearly_wins.items():
        if coalition not in mw_winning_coal_dict or mw_winning_coal_dict[coalition] != value:
            return False

    return True

In [40]:
pleasework = test_mvw_2('2002', winning_coal_dict, ger2002mv_dict)
pleasework

True